In [21]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### **Ensemble Retriever**

`EnsembleRetriever`는 여러 검색기(retriever)를 결합하여 더 강력한 검색 결과를 제공하는 LangChain의 기능입니다. <br>

**특징**

1. 여러 검색기 통합 <br>
    . 다양한 유형의 검색기(retriever)를 입력으로 반아 결과를 반환합니다. <br>
        
2. 결과를 재순위화 <br>
    . `Reciprocal Rank Fusion` 알고리즘을 사용하여 결과의 순위를 조정합니다. <br>
3. 하이브리드 검색<br>
    . `sparse retriever`와 `dense retriever`를 결합하여 사용합니다. <br>
    . `sparse retriever` - BM25 <br>
    . `dense retriever`  - 임베딩 유사도 검색


**장점**
- Sparse retriever: 키워드 기반 검색에 효과적 입니다.
- Dense retriever: 의미적 유사성 기반 검색에 효과적 입니다.


In [22]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

`EnsembleRetriever`를 초기화하여 `BM25Retriever`와 `FAISS` 검색기를 결합합니다.

In [23]:
doc_list = [
    "I like apples",
    "I like apple company",
    "I like apple's iphone",
    "Apple is my favorite company",
    "I like apple's ipad",
    "I like apple's macbook",
]

In [24]:
# bm25 retriever를 초기화 
bm25_retriever = BM25Retriever.from_texts(doc_list)

In [25]:
bm25_retriever.k = 1    # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

In [26]:
embedding = OpenAIEmbeddings(api_key=key)

In [27]:
# faiss retriever 초기화
faiss_vectorstore = FAISS.from_texts(
    doc_list,
    embedding,
)

In [28]:
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 1})

In [29]:
# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.8, 0.2],
)

`ensemble_retriever` 객체의 `get_relevant_documents()` 호출하여 관련성 높은 문서를 검색합니다.

In [30]:
query = "my favorite fruit is apple"
ensemble_result = ensemble_retriever.invoke(query)

In [31]:
print("[Ensemble Retriever]")

for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: Apple is my favorite company

Content: I like apples



In [32]:
query = "my favorite fruit is apple"
bm25_result = bm25_retriever.invoke(query)

In [33]:
print("[BM25 Retriever]")

for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

[BM25 Retriever]
Content: Apple is my favorite company



In [34]:
query = "my favorite fruit is apple"
faiss_result = faiss_retriever.invoke(query)

In [35]:
print("[FAISS Retriever]")

for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[FAISS Retriever]
Content: I like apples



In [36]:
query = "Apple company makes my favorite iphone"
ensemble_result = ensemble_retriever.invoke(query)

In [37]:
print("[Ensemble Retriever]")

for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: Apple is my favorite company

Content: I like apple's iphone



In [38]:
query = "Apple company makes my favorite iphone"
bm25_result = bm25_retriever.invoke(query)

In [39]:
print("[BM25 Retriever]")

for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

[BM25 Retriever]
Content: Apple is my favorite company



In [40]:
query = "Apple company makes my favorite iphone"
faiss_result = faiss_retriever.invoke(query)

In [41]:
print("[FAISS Retriever]")

for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[FAISS Retriever]
Content: I like apple's iphone

